In [ ]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars
using JLD2
using Serialization

In [ ]:
include("contractions.jl")
include("visualisation.jl")
include("preprocessing.jl")

In [ ]:
bunidata = loadsc("ply/torus.ply")

# rotate by 90 degrees around the x-axis
println("Get rotated lmao")
for vertex in bunidata.coords
    vertex[2], vertex[3] = vertex[2] * cos(π / 2) - vertex[3] * sin(π / 2), vertex[2] * sin(π / 2) + vertex[3] * cos(π / 2)
end

buni = initialContractedSimplicialComplex2D(bunidata)
include("visualisation.jl")
butthole = getborder(buni.contracted)
# convert Set{Tuple{Int64, Int64}}) to Array{GeometryBasics.Point{3,Float32}})
butthole_array = collect(butthole)
butthole = [GeometryBasics.Point3(buni.contracted.coords[butthole_array[i][1]]...) for i in 1:length(butthole_array)]
fillholes!(buni.contracted)
visualize(buni, butthole)